# MNIST Single Layer Network with TensorBoard

Note: This notebook is desinged to run with Python3 and GPU runtime.

![Python 3 and CPU runtime](https://raw.githubusercontent.com/enakai00/colab_tfbook/master/docs/imgs/runtime_gpu.png)

####[MST-01]
Import modules and set random seeds.

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import shutil

import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import initializers
from keras import callbacks
from keras.datasets import mnist

np.random.seed(20190228)
tf.set_random_seed(20190228)

plt.rcParams['axes.facecolor'] = 'white'

Using TensorFlow backend.


####[MST-02]
Download the MNIST dataset and store into NumPy arrays.

In [0]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(
                  (len(train_images), 784)).astype('float32') / 255
test_images = test_images.reshape(
                  (len(test_images), 784)).astype('float32') / 255
train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

####[MST-03]
Define a model with a single hidden layer.

In [3]:
model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=(28*28,),
                       kernel_initializer=initializers.TruncatedNormal(),
                       name='hidden'))
model.add(layers.Dense(10, activation='softmax', name='softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 1024)              803840    
_________________________________________________________________
softmax (Dense)              (None, 10)                10250     
Total params: 814,090
Trainable params: 814,090
Non-trainable params: 0
_________________________________________________________________


####[MST-04]
Compile the model using the Adam optimizer, and Cross entroy as a loss function.

In [0]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['acc'])

####[MST-05]
Train the model with the callbacks option to store training logs.

In [5]:
log_dir = '/tmp/log'
shutil.rmtree(log_dir, ignore_errors=True)
tensorboard_callback = callbacks.TensorBoard(log_dir=log_dir,
                                             histogram_freq=3)
history = model.fit(train_images, train_labels,
                    validation_data=(test_images, test_labels),
                    batch_size=128, epochs=10,
                    callbacks=[tensorboard_callback])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.2332 - acc: 0.9328 - val_loss: 0.1148 - val_acc: 0.9661
Epoch 2/10
60000/60000 [==============================] - 3s 45us/step - loss: 0.0887 - acc: 0.9737 - val_loss: 0.0848 - val_acc: 0.9717
Epoch 3/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.0562 - acc: 0.9833 - val_loss: 0.0713 - val_acc: 0.9787
Epoch 4/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.0372 - acc: 0.9892 - val_loss: 0.0677 - val_acc: 0.9791
Epoch 5/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.0270 - acc: 0.9925 - val_loss: 0.0580 - val_acc: 0.9822
Epoch 6/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.0198 - acc: 0.9941 - val_loss: 0.0594 - val_acc: 0.9819
Epoch 7/10
60000/60000 [==============================] - 3s 44us/step - loss: 0

####[MST-06]
Install ngrok to run TensorBoard on Colaboratory.

In [6]:
!curl -OL https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5237k  100 5237k    0     0  5026k      0  0:00:01  0:00:01 --:--:-- 5026k
Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


####[MST-07]
Start TensorBoard and prepare the connection URL.

In [7]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(log_dir)
)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://9293d626.ngrok.io
